# Example of using ChatGPT API for article summarization
Refer to: https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing


In [10]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Note: using Google CoLab


In [11]:
# install required packages
!pip install openai
!pip install gradio

In [12]:
# import the packages
import openai
import gradio as gr
import json
from typing import List, Dict, Tuple

client = openai.OpenAI()

# Check if you have set your ChatGPT API successfully
# You should see "Set ChatGPT API sucessfully!!" if nothing goes wrong.
try:
    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = [{'role':'user','content': "test"}],
            max_tokens=1,
    )
    print("Set ChatGPT API sucessfully!!")
except:
    print("There seems to be something wrong with your ChatGPT API. Please follow our demonstration in the slide to get a correct one.")

Set ChatGPT API sucessfully!!


In [14]:
# === Chatbot with history (Gradio + OpenAI Chat Completions) ===
!pip install -q --upgrade openai gradio

import os
from openai import OpenAI
import gradio as gr

# Use env var OPENAI_API_KEY (never hardcode)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

SYSTEM_PROMPT = (
    "You are a helpful, concise assistant. "
    "Answer clearly and use prior messages for context."
)

def build_messages(chat_history, user_input):
    """
    Convert Gradio chat history [(user, assistant), ...] + new user input
    into OpenAI 'messages' with a system prompt.
    """
    msgs = [{"role": "system", "content": SYSTEM_PROMPT}]
    for u, a in chat_history:
        if u:
            msgs.append({"role": "user", "content": u})
        if a:
            msgs.append({"role": "assistant", "content": a})
    msgs.append({"role": "user", "content": user_input})
    return msgs

def chat_complete(messages, model="gpt-3.5-turbo", temp=1.0, max_new_tokens=256):
    """
    Safe wrapper around client.chat.completions.create.
    - Uses max_completion_tokens (newer SDKs).
    - Omits temperature for models that only accept the default.
    - Falls back to gpt-3.5-turbo if needed.
    """
    kwargs = {
        "model": model,
        "messages": messages,
        "max_completion_tokens": max_new_tokens,
    }

    # Some newer models reject custom temperature; gpt-3.5-turbo is fine.
    if model in ["gpt-3.5-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini"]:
        kwargs["temperature"] = float(temp)

    try:
        resp = client.chat.completions.create(**kwargs)
        text = (resp.choices[0].message.content or "").strip()
        if text:
            return text
        # If empty (e.g., length finish because of reasoning tokens), retry bigger
        kwargs["max_completion_tokens"] = max(2 * max_new_tokens, 512)
        resp2 = client.chat.completions.create(**kwargs)
        return (resp2.choices[0].message.content or "").strip()
    except Exception:
        # Last resort: fallback
        kwargs["model"] = "gpt-3.5-turbo"
        kwargs["temperature"] = float(temp)
        resp3 = client.chat.completions.create(**kwargs)
        return (resp3.choices[0].message.content or "").strip()

def respond(user_input, chat_history, model, temp):
    if not user_input.strip():
        return chat_history
    messages = build_messages(chat_history, user_input)
    reply = chat_complete(messages, model=model, temp=temp, max_new_tokens=256)
    chat_history = chat_history + [(user_input, reply)]
    return chat_history

def reset_chat():
    return []

# ---------- Gradio UI ----------
with gr.Blocks() as demo:
    gr.Markdown("## Part 4: ChatBot (with context)")

    chatbot = gr.Chatbot(height=420)  # shows all turns
    with gr.Row():
        model_dd = gr.Dropdown(
            label="Model",
            choices=["gpt-3.5-turbo", "gpt-4o", "gpt-4o-mini"],
            value="gpt-3.5-turbo",
            scale=2,
        )
        temp = gr.Slider(0.0, 1.5, value=1.0, step=0.1, label="Temperature", scale=1)

    msg = gr.Textbox(label="Input", placeholder="Ask me anything…", lines=2)
    with gr.Row():
        send = gr.Button("Send", variant="primary")
        clear = gr.Button("Reset")

    # Wire events
    send.click(fn=respond, inputs=[msg, chatbot, model_dd, temp], outputs=chatbot)
    send.click(fn=lambda: "", outputs=msg)  # clear input after send
    clear.click(fn=reset_chat, outputs=chatbot)

demo.launch(debug=True)



/tmp/ipython-input-1163845991.py:78: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)  # shows all turns


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b701d064713d12d696.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b701d064713d12d696.gradio.live
